
# 2.2-kW PMSM, diode bridge

This example simulates sensorless vector control of a 2.2-kW PMSM drive. 


Imports.



In [ ]:
from motulator import model, control
from motulator import BaseValues, plot

Compute base values based on the nominal values (just for figures).



In [ ]:
base = BaseValues(
    U_nom=370, I_nom=4.3, f_nom=75, tau_nom=14, P_nom=2.2e3, n_p=3)

Configure the system model.



In [ ]:
machine = model.sm.SynchronousMachine(
    n_p=3, R_s=3.6, L_d=.036, L_q=.051, psi_f=.545)
mechanics = model.Mechanics(J=.015)
converter = model.Inverter(u_dc=540)
mdl = model.sm.Drive(machine, mechanics, converter)
# mdl.pwm = model.CarrierComparison()  # Enable the PWM model

Configure the control system.



In [ ]:
par = control.sm.ModelPars(
    n_p=3, R_s=3.6, L_d=.036, L_q=.051, psi_f=.545, J=.015)
ref = control.sm.CurrentReferencePars(par, w_m_nom=base.w, i_s_max=1.5*base.i)
ctrl = control.sm.VectorCtrl(par, ref, T_s=250e-6, sensorless=True)

Set the speed reference and the external load torque.



In [ ]:
# Speed reference
ctrl.w_m_ref = lambda t: (t > .2)*2*base.w

# External load torque
mdl.mechanics.tau_L_t = lambda t: (t > .8)*.7*base.tau_nom

Create the simulation object and simulate it.



In [ ]:
# Simulate the system and plot results in per-unit values
sim = model.Simulation(mdl, ctrl)
sim.simulate(t_stop=1.4)
plot(sim, base)